In [1]:
import sys
sys.path.append("/Users/avgalichin/Documents/kaggle/Jigsaw2022/Jigsaw18/src")
from pathlib import Path
import pandas as pd

from transformers import AutoTokenizer

from data_utils import clean, encode_multilabel, get_folds, encode_df

In [2]:
data_dir = Path("/Users/avgalichin/Documents/kaggle/Jigsaw2022/Jigsaw18/dataset")

train_path = data_dir / "raw_data" / "train.csv"
test_path = data_dir / "raw_data" / "test.csv"
test_labels_path = data_dir / "raw_data" / "test_labels.csv"

In [3]:
class_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [4]:
train_data = pd.read_csv(train_path)
print(train_data.shape)
train_data.head()

(159571, 8)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
test_data = pd.read_csv(test_path)
test_labels = pd.read_csv(test_labels_path)

test_data = test_data.merge(test_labels, on='id')
test_data = test_data[test_data[class_columns].sum(axis=1) != -6]  # all -1 is invalid comment
test_data.reset_index(drop=True, inplace=True)

print(test_data.shape)
test_data.head()

(63978, 8)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0001ea8717f6de06,Thank you for understanding. I think very high...,0,0,0,0,0,0
1,000247e83dcc1211,:Dear god this site is horrible.,0,0,0,0,0,0
2,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0,0,0,0,0,0
3,0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0,0,0,0,0,0
4,00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",0,0,0,0,0,0


In [6]:
# clean data 

train_data = clean(train_data.copy(), col_name='comment_text', min_len=10)
test_data = clean(test_data.copy(), col_name='comment_text', min_len=10)

print(train_data.shape), print(test_data.shape)

/Users/avgalichin/Documents/kaggle/Jigsaw2022/Jigsaw18/src/data_utils.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, col_name] = df[col_name].str.replace('https?://\S+|www\.\S+', ' social medium ')
/Users/avgalichin/Documents/kaggle/Jigsaw2022/Jigsaw18/src/data_utils.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, col_name] = df[col_name].str.replace('\s+', ' ')


(159165, 8)
(63804, 8)


(None, None)

#### Further is splitting test on val and test

In [7]:
# multilabel encoding -> labels for stratified split

id2fold_path = data_dir / "proc_data" / "id2fold.csv"

try:
    id2fold = pd.read_csv(id2fold_path)
    test_data = test_data.merge(id2fold, on='id')
except FileNotFoundError:
    print("No id2label file found. Creating new split")

    test_data = encode_multilabel(test_data.copy(), columns=class_columns)
    test_data.loc[:, 'label'] = test_data.label.apply(int)
    print(test_data.label.nunique())

    # split by folds
    n_folds = 5

    test_data = get_folds(test_data.copy(), n_folds=n_folds)

print(test_data.shape)
test_data.head()

(63804, 9)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,fold
0,0001ea8717f6de06,Thank you for understanding. I think very high...,0,0,0,0,0,0,4
1,000247e83dcc1211,:Dear god this site is horrible.,0,0,0,0,0,0,0
2,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0,0,0,0,0,0,0
3,0003e1cccfd5a40a,""" It says it right there that it IS a type. Th...",0,0,0,0,0,0,1
4,00059ace3e3e9a53,""" == Before adding a new product to the list, ...",0,0,0,0,0,0,4


#### Encoding (Carefully: Based on the model you chose; different tokenizers or diff. models)

In [21]:
tokenizer_type = 'bert-base-cased'  # change if needed

tokenizer = AutoTokenizer.from_pretrained(tokenizer_type)

In [12]:
train_data = encode_df(train_data.copy(), tokenizer,
                       col_name='comment_text', max_length=256, drop=True)
test_data = encode_df(test_data.copy(), tokenizer,
                      col_name='comment_text', max_length=256, drop=True)

100%|██████████| 159165/159165 [01:01<00:00, 2607.49it/s]
/Users/avgalichin/Documents/kaggle/Jigsaw2022/Jigsaw18/src/data_utils.py:78: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(col_name, 1, inplace=True)
100%|██████████| 63804/63804 [00:23<00:00, 2707.84it/s]


In [13]:
train_data.to_csv(data_dir / 'proc_data' / 'bert_train_data.csv', index=False)
# test_data.to_csv(data_dir / 'proc_data' / 'test_data.csv', index=False)

In [14]:
# split on val/test
val_fold = 4

val_data = test_data[test_data.fold == val_fold]
test_data = test_data[test_data.fold != val_fold]

print(val_data.shape)
print(test_data.shape)

(12760, 10)
(51044, 10)


In [15]:
test_data.to_csv(data_dir / 'proc_data' / 'bert_test_data.csv', index=False)
val_data.to_csv(data_dir / 'proc_data' / 'bert_val_data.csv', index=False)